In [16]:
from sklearn.metrics.cluster import adjusted_rand_score
from kshape.core import kshape, zscore
import numpy as np
import random
import pickle

import warnings
warnings.filterwarnings("ignore")

# Cluster Algorithms

In [17]:
def ComputeSSE(dataset,labels,centroids):
    
    SSE = 0
    for i in set(labels):
        for j in range(len(labels)):
            if(i==labels[j]):
                SSE += calc_distance(dataset[j], centroids[i])**2
    
    return SSE

## K-MDTSC

In [3]:
def calc_distance(s1, s2):
    
    if(len(s1)==0 or len(s2)==0): return 0
    
    d = 0
    
    for dimension in range(len(s1)):

        for i in range(len(s1[dimension])):
            d += ((s1[dimension][i]-s2[dimension][i])**2)

    d=d**0.5

    return d

def assign_to_clusters(c,s1):
    
    label = -1
    min_dist = 100000000000
    
    for i in range(len(c)):
        ci = c[i]
        tmp_dist = calc_distance(s1, ci)
        if(tmp_dist < min_dist): 
            min_dist = tmp_dist
            label=i
    
    return label

def compute_centroid(points):
   
    if(len(points)==0): return []
    
    centroid = [[0 for i in range(len(points[0][metrics]))] for metrics in range(len(points[0]))]
    
    for point in points:
        for metric in range(len(point)):
            samples = point[metric]
            for i in range(len(samples)):
                centroid[metric][i]+=samples[i]
    
    for metric in range(len(centroid)):
        for i in range(len(centroid[metric])):
            centroid[metric][i]/=len(points)
    
    
    return centroid


def assign_one_point(cluster_points,empty_cluster):
    
    most_popular = 0
    for cluster in cluster_points:
        if(len(cluster_points[cluster])>len(cluster_points[most_popular])):
            most_popular = cluster
            
    farest = -1
    farest_point = 0
    ci = compute_centroid(cluster_points[most_popular])
    for i in range(len(cluster_points[most_popular])):
        s1 = cluster_points[most_popular][i]
        tmp_dist = calc_distance(s1, ci)
        if(tmp_dist>farest):
            farest=tmp_dist
            farest_point = i
    
    
    s1 = cluster_points[most_popular][farest_point]
    cluster_points[empty_cluster].append(s1)
    
    del cluster_points[most_popular][farest_point]
    
    return


def create_clusters(dataset, nclusters):

    current_centroids = random.sample(dataset, nclusters)
    current_distance = -1 

    iterations = 100
    labels = []
    iteration=0
    for iteration in range(iterations):

        cluster_points = {cluster:[] for cluster in range(nclusters)}
        labels = []        
        for s1 in dataset:
            cluster = assign_to_clusters(current_centroids,s1)
            cluster_points[cluster].append(s1)    
            labels.append(cluster)

        for i in range(nclusters):
            if(len(cluster_points[i])==0):
                assign_one_point(cluster_points,i)

        new_centroids = []
        dist = 0 
        for i in range(nclusters):
            centroidi = compute_centroid(cluster_points[i])
            new_centroids.append(centroidi)
            
            dist +=calc_distance(current_centroids[i], new_centroids[i])

        if(dist==0): 
            return labels,new_centroids,cluster_points

        current_distance=dist
        current_centroids = new_centroids
    
    return labels, current_centroids, cluster_points

In [4]:
def run_cluster(dataset,families,attempts,minc,maxc):
        
    Results = {}
    for nclusters in range(minc,maxc):
        Results[nclusters] = []
        for attempt in range(attempts):
            labels,centroids, cluster_points = create_clusters(dataset,nclusters)
            
            SSE = ComputeSSE(dataset,labels,centroids)
            Rand = adjusted_rand_score(labels, [i%families for i in range(len(labels))])
            
            Results[nclusters].append({"labels": labels.copy(),"centroids":centroids.copy(),"SSE":SSE,"RandIndex":Rand})

    return Results

## k-Shape

In [5]:
def denormalize_centroids(dataset,clusters):

    denorm_centroids = {i:[] for i in range(len(clusters))}

    nc = 0
    for cluster in clusters:
        centroid = cluster[0]
        components = {}
        for dimension in range(len(dataset[0])):
            components[dimension] = [[] for _ in range(len(dataset[0][dimension]))]
            
        for pointid in cluster[1]:
            point = dataset[pointid]
            for dimension in range(len(point)):
                samples = point[dimension]
                for i in range(len(samples)):
                    components[dimension][i].append(samples[i])

        for dimension in range(len(dataset[0])):     
            nsamples = len(dataset[0][dimension])
            centroid_dimnension = centroid[dimension*nsamples:(dimension+1)*nsamples].copy()
            for k in range(len(components[dimension])):
                centroid_dimnension[k] = centroid_dimnension[k]*np.std(components[dimension][k])+np.mean(components[dimension][k])
        
            denorm_centroids[nc].append(centroid_dimnension) 
        nc+=1
    return denorm_centroids

def get_labels(dataset_kshape,clusters):
    
    labels = [0 for _ in range(len(dataset_kshape))]
    nc=0
    for cluster in clusters:
        for pointid in cluster[1]: labels[pointid]=nc
        nc+=1
    return labels

In [6]:
def run_k_shape(dataset,families,attempts,minc,maxc):

    dataset_kshape = []
    for p in dataset:
        pg = []
        for i in range(len(p)):
            pg =pg + list(p[i]) 
        dataset_kshape.append(pg)    
        
    Results = {}
    
    for nclusters in range(minc,maxc):
        Results[nclusters]=[]
        for attempt in range(attempts):
            clusters = kshape(dataset_kshape, nclusters)
            centroids = denormalize_centroids(dataset,clusters)
            labels = get_labels(dataset_kshape,clusters)

            SSE = ComputeSSE(dataset,labels,centroids)
            Rand = adjusted_rand_score(labels, [i%families for i in range(len(labels))])

            Results[nclusters].append({"labels": labels.copy(),"centroids":centroids.copy(),"SSE":SSE,"RandIndex":Rand})
                           
    return Results

# Synthetic Data

In [18]:
def SinNoise(x,period=1,noise=0.1):
    return np.sin(x*period) + np.random.normal(scale=noise, size=len(x))

def CosNoise(x,period=1,noise=0.1):
    return np.cos(x*period) + np.random.normal(scale=noise, size=len(x))

def generate_points(npoints, families=4, dimensions=2, noise=0.1):
    
    points = []

    family_config_periods_dimension = {}
    for i in range(families):
        periods_dimension = [1+int(i/4),1+int(i/4)]+\
        [random.uniform(2+int(i/4),10+int(i/4)) for _ in range(dimensions-2)]
        
        family_config_periods_dimension[i] = periods_dimension
    
    for i in range(npoints):
        dimensions_values=[]
        family = i%families
        config = family_config_periods_dimension[family][-1]
        x = np.linspace(0, 2*np.pi)
        if(config==0):
            for j in range(dimensions):
                if(j%2==0):
                    si1 = SinNoise(x,period=family_config_periods_dimension[family][j],noise=noise)
                    dimensions_values.append(si1)
                else:
                    si2 = CosNoise(x,period=family_config_periods_dimension[family][j],noise=noise)
                    dimensions_values.append(si2)
        if(config==1):
            for j in range(dimensions):
                if(j%2==0):
                    si1 = CosNoise(x,noise=noise,period=family_config_periods_dimension[family][j])
                    dimensions_values.append(si1)
                else:
                    si2 = SinNoise(x,noise=noise,period=family_config_periods_dimension[family][j])
                    dimensions_values.append(si2)
        if(config==2):
            for j in range(dimensions):
                if(j%2==0):
                    si1 = SinNoise(x,noise=noise,period=family_config_periods_dimension[family][j])
                    dimensions_values.append(si1)
                else:
                    si2 = SinNoise(x,noise=noise,period=family_config_periods_dimension[family][j]) 
                    dimensions_values.append(si2)
        if(config==3):
            for j in range(dimensions):
                if(j%2==0):
                    si1 = CosNoise(x,noise=noise,period=family_config_periods_dimension[family][j])
                    dimensions_values.append(si1)
                else:
                    si2 = CosNoise(x,noise=noise,period=family_config_periods_dimension[family][j]) 
                    dimensions_values.append(si2)
        
        points.append(dimensions_values)

    return points

# Run Synthetic Analysis

#### Preliminary

In [9]:
#Number of multi-dimensional time-series 
Points = 200
#Numer of runs for each clustering algorithm
attempts = 10

#Load the Dataset used to run the paper experiment 
dataset = pickle.load(open("result/Dataset_Synthetic.pkl","rb"))
#Decomment to create a new random dataset
#dataset = generate_points(Points, families=4, dimensions=2, noise=0.1)

ResultKMDTSC = run_cluster(dataset,families=4, attempts=attempts, minc=1, maxc=11)
ResultkShape = run_k_shape(dataset,families=4, attempts=attempts, minc=1, maxc=11)

pickle.dump(dataset, open("result/Dataset_Synthetic.pkl","wb"))
pickle.dump(ResultKMDTSC, open("result/KMDTSC_Synthetic.pkl","wb")) 
pickle.dump(ResultkShape, open("result/kShape_Synthetic.pkl","wb")) 

#### Perturbation stability

In [13]:
ResultsKMDTSC = {}
ResultskShape = {}

Datasets = {}
#Load the Datasets used to run the paper experiment 
Datasets = pickle.load(open("result/Datasets_Perturbation.pkl","rb"))

#Number of multi-dimensional time-series 
Points = 200
#Numer of runs for each clustering algorithm
attempts = 10

for i in range(3,5):
    for noisei in np.arange(0,2.1,0.1).round(decimals=1):
        
        dataset = Datasets[noisei]
        #Decomment to create a new random dataset
        #dataset = generate_points(Points, families=4, dimensions=2, noise=noisei)

        ResultKMDTSC = run_cluster(dataset,families=4, attempts=attempts, minc=4, maxc=5)
        ResultkShape = run_k_shape(dataset,families=4, attempts=attempts, minc=4, maxc=5)

        #Datasets[noisei] = dataset
        ResultsKMDTSC[noisei] = ResultKMDTSC
        ResultskShape[noisei] = ResultkShape

    pickle.dump(Datasets, open("result/Datasets_Perturbation.pkl","wb"))
    pickle.dump(ResultsKMDTSC, open("result/KMDTSC_Perturbation.pkl","wb")) 
    pickle.dump(ResultskShape, open("result/kShape_Perturbation.pkl","wb")) 

#### Family stability

In [14]:
ResultsKMDTSC = {}
ResultskShape = {}

Datasets = {}
#Load the Datasets used to run the paper experiment 
Datasets = pickle.load(open("result/Datasets_Family.pkl","rb"))

#Number of multi-dimensional time-series 
Points = 200
#Numer of runs for each clustering algorithm
attempts = 10
for i in range(4):

    for familyi in range(2,11):
        dataset = Datasets[familyi]
        #Decomment to create a new random dataset
        #dataset = generate_points(Points, families=familyi, dimensions=2, noise=0.1)

        ResultKMDTSC = run_cluster(dataset, families=familyi, attempts=attempts, minc=familyi, maxc=familyi+1)
        ResultkShape = run_k_shape(dataset, families=familyi, attempts=attempts, minc=familyi, maxc=familyi+1)

        Datasets[familyi] = dataset
        ResultsKMDTSC[familyi] = ResultKMDTSC
        ResultskShape[familyi] = ResultkShape

    pickle.dump(Datasets, open("result/Datasets_Family.pkl","wb"))
    pickle.dump(ResultsKMDTSC, open("result/KMDTSC_Family.pkl","wb")) 
    pickle.dump(ResultskShape, open("result/kShape_Family.pkl","wb")) 

#### Dimension stability

In [ ]:
ResultsKMDTSC = {}
ResultskShape = {}

Datasets = {}
#Load the Datasets used to run the paper experiment 
Datasets = pickle.load(open("result/Datasets_Dimension.pkl","rb"))

#Number of multi-dimensional time-series 
Points = 200
#Numer of runs for each clustering algorithm
attempts = 10
noisei = 0.1
for dimensions in range(2,11):

    dataset = Datasets[dimensions]
    #Decomment to create a new random dataset
    #dataset = generate_points(Points, families=4, dimensions=dimensions, noise=0.1)

    ResultKMDTSC = run_cluster(dataset, families=4, attempts=attempts, minc=4, maxc=5)
    ResultkShape = run_k_shape(dataset, families=4, attempts=attempts, minc=4, maxc=5)

    Datasets[dimensions] = dataset
    ResultsKMDTSC[dimensions] = ResultKMDTSC
    ResultskShape[dimensions] = ResultkShape

pickle.dump(Datasets, open("result/Datasets_Dimension.pkl","wb"))
pickle.dump(ResultsKMDTSC, open("result/KMDTSC_Dimension.pkl","wb")) 
pickle.dump(ResultskShape, open("result/kShape_Dimension.pkl","wb")) 